In [23]:
import pyreadr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from sklearn.linear_model import LinearRegression as lm
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

embed = lambda x, d : np.hstack([x.shift(i) for i in range(d)])[d - 1:] # R's embed(x, d)

Y = pyreadr.read_r('rawdata.rda')['dados']
nprev=132

In [2]:
indice = 0
lag = 1
alpha = 1
type = 'lasso'

In [7]:
    coef = np.empty((nprev, 21 + (Y.shape[1] - 1) * 4))*np.nan
    pred = np.empty(nprev)*np.nan

In [ ]:
for i in range(nprev):
        window = Y.iloc[i:len(Y)-(nprev-i), [indice]] # The last obs. is not inclued in the last iter.
        model, pred[i], coef[i] = runlasso(window, lag)
    

In [ ]:
window = 

In [65]:
pca = PCA()
SCALE = scale(Y, with_mean=True, with_std=False)
pca.fit(SCALE)

Y2 = pd.concat([Y, pd.DataFrame(pca.transform(SCALE)[:, :4], index=Y.index)], axis=1)
aux = embed(Y2, lag + 4)

y = aux[:, indice]
X = np.delete(aux, range(Y2.shape[1] * lag), axis=1)

In [89]:
if lag == 1:
    out = aux[-1, :X.shape[1]]
else:
    out = np.delete(aux, range(Y2.shape[1] * (lag-1)), axis=1)
    out = out[-1, :X.shape[1]]

In [90]:
    y = y[:len(y)-lag+1]
    X = X[:len(X)-lag+1]

In [ ]:
runlasso=function(Y,indice,lag,alpha=1,type="lasso"){
  comp=princomp(scale(Y,scale=FALSE))
  Y2=cbind(Y,comp$scores[,1:4])
  aux=embed(Y2,4+lag)
  y=aux[,indice]
  X=aux[,-c(1:(ncol(Y2)*lag))]  
  
  if(lag==1){
    X.out=tail(aux,1)[1:ncol(X)]  
  }else{
    X.out=aux[,-c(1:(ncol(Y2)*(lag-1)))]
    X.out=tail(X.out,1)[1:ncol(X)]
  }
  
  y = y[1:(length(y)-lag+1)]
  X = X[1:(nrow(X)-lag+1),]
  
  model=ic.glmnet(X,y,alpha = alpha)
  coef=model$coef
  if(type=="adalasso"){
    penalty=(abs(coef[-1])+1/sqrt(length(y)))^(-1)
    model=ic.glmnet(X,y,penalty.factor = penalty,alpha=alpha)
  }
  
  if(type=="fal"){
    taus=c(seq(0.1,1,0.1),1.25,1.5,2,3,4,5,7,10)
    alphas=seq(0,1,0.1)
    bb=Inf
    for(alpha in alphas){
      m0=ic.glmnet(X,y,alpha = alpha)
      coef=m0$coef
      for(tau in taus){
        penalty=(abs(coef[-1])+1/sqrt(length(y)))^(-tau)
        m=ic.glmnet(X,y,penalty.factor = penalty)
        crit=m$bic
        if(crit<bb){
          model=m
          bb=crit
        }
      }
    }
  }
  pred=predict(model,X.out)
  
  return(list("model"=model,"pred"=pred))
}


lasso.rolling.window=function(Y,nprev,indice=1,lag=1,alpha=1,type="lasso"){
  
  save.coef=matrix(NA,nprev,21+ncol(Y[,-1])*4)
  save.pred=matrix(NA,nprev,1)
  for(i in nprev:1){
    Y.window=Y[(1+nprev-i):(nrow(Y)-i),]
    lasso=runlasso(Y.window,indice,lag,alpha,type)
    save.coef[(1+nprev-i),]=lasso$model$coef
    save.pred[(1+nprev-i),]=lasso$pred
    cat("iteration",(1+nprev-i),"\n")
  }
  
  real=Y[,indice]
  plot(real,type="l")
  lines(c(rep(NA,length(real)-nprev),save.pred),col="red")
  
  rmse=sqrt(mean((tail(real,nprev)-save.pred)^2))
  mae=mean(abs(tail(real,nprev)-save.pred))
  errors=c("rmse"=rmse,"mae"=mae)
  
  return(list("pred"=save.pred,"coef"=save.coef,"errors"=errors))
}


## == ##
runpols=function(Y,indice,lag,coef){
  
  comp=princomp(scale(Y,scale=FALSE))
  Y2=cbind(Y,comp$scores[,1:4])
  aux=embed(Y2,4+lag)
  y=aux[,indice]
  X=aux[,-c(1:(ncol(Y2)*lag))]  
  
  if(lag==1){
    X.out=tail(aux,1)[1:ncol(X)]  
  }else{
    X.out=aux[,-c(1:(ncol(Y2)*(lag-1)))]
    X.out=tail(X.out,1)[1:ncol(X)]
  }
  
  respo=X[,which(coef[-1]!=0)]
  if(length(respo)==0){
    model=lm(y ~ 1)
  }else{
    model=lm(y ~ respo)
  }
  
  pred=c(1,X.out[which(coef[-1]!=0)])%*%coef(model)
  
  return(list("model"=model,"pred"=pred))
}


pols.rolling.window=function(Y,nprev,indice=1,lag=1,coef){
  
  save.pred=matrix(NA,nprev,1)
  for(i in nprev:1){
    Y.window=Y[(1+nprev-i):(nrow(Y)-i),]
    m=runpols(Y.window,indice,lag,coef[(1+nprev-i),])
    save.pred[(1+nprev-i),]=m$pred
    cat("iteration",(1+nprev-i),"\n")
  }
  
  real=Y[,indice]
  plot(real,type="l")
  lines(c(rep(NA,length(real)-nprev),save.pred),col="red")
  
  rmse=sqrt(mean((tail(real,nprev)-save.pred)^2))
  mae=mean(abs(tail(real,nprev)-save.pred))
  errors=c("rmse"=rmse,"mae"=mae)
  
  return(list("pred"=save.pred,"errors"=errors))
}


In [92]:
import numpy as np
from sklearn.datasets import load_boston, load_diabetes
from sklearn.linear_model import _coordinate_descent as cd

import matplotlib.pyplot as plt

import glmnet

boston = load_boston()
diabetes = load_diabetes()
X, y = boston['data'], boston['target']
# X, y = diabetes['data'], diabetes['target']

lambdas = np.logspace(np.log10(1e-4), np.log10(35), 100)

ModuleNotFoundError: No module named 'glmnet'